In [1]:
'''
$ git clone https://github.com/BingHongLi/line_chat_bot_tutorial.git
'''

'''
$ docker exec -it line-chat-bot-jupyter /bin/bash

$ pip install pandas

$ pip install scikit-learn

$ pip install mysql-connector-python

$ pip install keras

$ pip install TensorFlow

跳出container(鍵盤操作): Ctrl + P + Q
'''

'\n$ docker exec -it line-chat-bot-jupyter /bin/bash\n\n$ pip install pandas\n\n$ pip install scikit-learn\n\n$ pip install mysql-connector-python\n\n$ pip install keras\n\n$ pip install TensorFlow\n\n跳出container(鍵盤操作): Ctrl + P + Q\n'

In [2]:
"""

啟用伺服器基本樣板

"""

# 引用Web Server套件
from flask import Flask, request, abort

# 從linebot 套件包裡引用 LineBotApi 與 WebhookHandler 類別
from linebot import (
    LineBotApi, WebhookHandler
)

# 引用無效簽章錯誤
from linebot.exceptions import (
    InvalidSignatureError
)

# 載入json處理套件
import json

# 將消息模型，文字收取消息與文字寄發消息 引入
from linebot.models import (
    MessageEvent, TextMessage, TextSendMessage,ImageMessage
)

# 載入基礎設定檔
secretFileContentJson=json.load(open("./line_secret_key",'r'))
print(secretFileContentJson.get("channel_access_token"))
print(secretFileContentJson.get("secret_key"))
print(secretFileContentJson.get("self_user_id"))
server_url = secretFileContentJson.get("server_url")

# 設定Server啟用細節
app = Flask(__name__,static_url_path = "/images" , static_folder = "./images/")

# 生成實體物件
line_bot_api = LineBotApi(secretFileContentJson.get("channel_access_token"))
handler = WebhookHandler(secretFileContentJson.get("secret_key"))

# 啟動server對外接口，使Line能丟消息進來
@app.route("/", methods=['POST'])
def callback():
    # get X-Line-Signature header value
    signature = request.headers['X-Line-Signature']

    # get request body as text
    body = request.get_data(as_text=True)
    app.logger.info("Request body: " + body)

    # handle webhook body
    try:
        handler.handle(body, signature)
    except InvalidSignatureError:
        abort(400)

    return 'OK'

Hlhy/OakZWX/icqhCzMSOetHgPex89AcgVKJVAprtvx3pHC0WXIMk4sZmTy0cslymmKGzwKLp5YV8rAl5Tgge87gNm9VzmoZiwkyL9rcMRgEzKo92QZpJzp3vGpr5UKjlEzUHx8XUILszM33ag0BwAdB04t89/1O/w1cDnyilFU=
37e772fa7d48063a4cfd8669c6107046
U873be787f1bc8a4322527ab2310bc0ab


In [3]:
#經營地的tag，拿到手
users_cache = {}

#user_profile使用者代號
#prop_key使用者回傳值
#users_cache回傳值字典

def del_user_prop(user_profile, prop_key):
    if user_profile.user_id in users_cache and prop_key in users_cache[user_profile.user_id]:
        del users_cache[user_profile.user_id][prop_key]
#創建第一個使用這代號
def put_user_prop(user_profile, prop_key, prop_val):
    if not user_profile.user_id in users_cache:
        users_cache[user_profile.user_id] = {}
    #?????????????   
    users_cache[user_profile.user_id][prop_key] = prop_val
#回傳值，並結束後重新開始
def get_user_prop(user_profile, prop_key):
    #使用者代號在回傳值字典裏面 和 回傳值在使用者者代號裡面  回傳 使用者代號及回傳值
    if user_profile.user_id in users_cache and prop_key in users_cache[user_profile.user_id]:
        return users_cache[user_profile.user_id][prop_key]
    #結束後變成空值
    return None

In [4]:
#經營地的tag，拿到手
users_cache_food = {}

#user_profile使用者代號
#prop_key使用者回傳值
#users_cache回傳值字典

def del_user_prop_food(user_profile_food, prop_key):
    if user_profile_food.user_id in users_cache_food and prop_key in users_cache_food[user_profile_food.user_id]:
        del users_cache_food[user_profile_food.user_id][prop_key]
#創建第一個使用這代號
def put_user_prop_food(user_profile_food, prop_key, prop_val):
    if not user_profile_food.user_id in users_cache_food:
        users_cache_food[user_profile_food.user_id] = {}
    #?????????????   
    users_cache_food[user_profile_food.user_id][prop_key] = prop_val
#回傳值，並結束後重新開始
def get_user_prop_food(user_profile_food, prop_key):
    #使用者代號在回傳值字典裏面 和 回傳值在使用者者代號裡面  回傳 使用者代號及回傳值
    if user_profile_food.user_id in users_cache_food and prop_key in users_cache_food[user_profile_food.user_id]:
        return users_cache_food[user_profile_food.user_id][prop_key]
    #結束後變成空值
    return None

In [5]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import mysql.connector
import traceback

def recommend_camp_sites(user_contents, num):
    ret = None
    conn = None
    try:
        cfg = {
            "host": "35.229.201.139",
            "port": 3306,
            "database": "camp_db",
            "user": "iii",
            "password": "Qwer_0987"
        }
        conn = mysql.connector.connect(**cfg)
        df = pd.read_sql("SELECT DISTINCT l.camp_title,l.camp_site,l.addr,l.latlong,l.location,l.style,l.tags,l.fb_rating,w.url FROM camp_list l JOIN camp_webs w ON l.camp_title = w.camp_title", conn)
        train_series = df["tags"] # Series
        test_series = pd.Series(user_contents) # Series
#         tokenizer = TfidfVectorizer(analyzer="word", ngram_range=(1, 2), min_df=0, stop_words="english")
        tokenizer = TfidfVectorizer()
        train_tfidf = tokenizer.fit_transform(train_series)
        test_tfidf = tokenizer.transform(test_series)
        df_cs = pd.DataFrame(cosine_similarity(test_tfidf, train_tfidf), columns=train_series.index)
        df_cs["ix_test"] = test_series.index
        # pd.melt(): http://shzhangji.com/cnblogs/2017/09/30/pandas-and-tidy-data/
        # frame: 需要處理的數據框；
        # id_vars: 保持原樣的數據列；
        # value_vars: 需要被轉換成變量值的數據列；
        # var_name: 轉換後變量的列名；
        # value_name: 數值變量的列名。
        score = pd.melt(
            df_cs,
            id_vars = "ix_test",
            var_name = "ix_train",
            value_name = "score"
        )
        df["score"] = score["score"]
        # score遞增排序並取出指定的筆數
        df = df.sort_values(by="score")[:-(num+1):-1]
        ret = df[["camp_title", "camp_site", "tags", "score","addr","url"]]
    except Exception as e:
        print("Error: {}".format(e))
        traceback.print_exc()
    finally:
        if conn:
            conn.close()
    return ret

In [6]:
from keras.models import load_model
from keras.preprocessing import image
import numpy as np

def cnn_itemindntify(img_path):
    predict = None
    items = ['吊籃', '避雷帽', '豬尾巴']
    model = load_model("./cnn.h5")
    img = image.load_img(img_path, target_size=(224, 224))
    x = image.img_to_array(img)
    x = np.expand_dims(x, axis=0) / 255  # 圖片: 處理0~1
    predict = items[model.predict_classes(x)[0]]
    return predict

Using TensorFlow backend.


In [7]:
'''

製作文字與圖片的教學訊息

'''
# 將消息模型，文字收取消息與文字寄發消息 引入
from linebot.models import (
    MessageEvent, TextMessage, TextSendMessage, ImageSendMessage
)

# 消息清單
reply_message_list = [
TextSendMessage(text="---------------------\n使用說明:\n---------------------\n一、營地推薦\n(點選關鍵字送出，推薦您適合的營地。)\n    \n二、拍照辨識\n(拍照立即辨識物品，並搜尋在哪裡買。)\n\n三、搜尋營地\n(選擇範圍及類型，推薦您適合的營地。)\n\n四、食譜推薦\n(選擇食材，推薦您適合的料理。)\n\n五、待確定\n\n\n六、關於我們\n(由任何問題，歡迎留言或發E-mail給我們。)")
]

In [8]:
'''

撰寫用戶關注時，我們要處理的商業邏輯

1. 取得用戶個資，並存回伺服器
2. 把先前製作好的自定義菜單，與用戶做綁定
3. 回應用戶，歡迎用的文字消息與圖片消息

'''


# 載入Follow事件
from linebot.models.events import (
    FollowEvent
)

# 載入requests套件
import requests


# 告知handler，如果收到FollowEvent，則做下面的方法處理
@handler.add(FollowEvent)
def reply_text_and_get_user_profile(event):
    
    # 取出消息內User的資料
    user_profile = line_bot_api.get_profile(event.source.user_id)
    print("user_profile:", user_profile)
        
     # 將用戶資訊存在檔案內
    with open("./users.txt", "a") as myfile:
        myfile.write(json.dumps(vars(user_profile),sort_keys=True))
        myfile.write('\r\n')
        
        
    # 將菜單綁定在用戶身上
    linkRichMenuId = secretFileContentJson.get("rich_menu_id")
    linkMenuEndpoint = 'https://api.line.me/v2/bot/user/%s/richmenu/%s' % (event.source.user_id, linkRichMenuId)
    linkMenuRequestHeader = {'Content-Type':'image/jpeg','Authorization':'Bearer %s' % secretFileContentJson["channel_access_token"]}
    lineLinkMenuResponse = requests.post(linkMenuEndpoint,headers=linkMenuRequestHeader)
    
     # 回覆文字消息與圖片消息
    line_bot_api.reply_message(
         event.reply_token,
         reply_message_list
     )

In [ ]:
'''
偏好調查(CarouselTemplate)
'''

from linebot.models import (
    TemplateSendMessage,
    CarouselTemplate, CarouselColumn,
    PostbackAction, MessageAction, URIAction
)


'''
Reference: https://github.com/line/line-bot-sdk-python#templatesendmessage---buttonstemplate

'''

survey_carousel_template_message = TemplateSendMessage(
    alt_text='喜歡就按讚',
    template=CarouselTemplate(
        columns=[
            CarouselColumn(
                thumbnail_image_url='https://%s/images/螢火蟲.jpg' % server_url,
#                 title='this is menu1',
                text='螢火蟲,夜景',
                actions=[
                    PostbackAction(
                        label='讚',
#                         text='[::survey_like:]螢火蟲,夜景',
                        data='survey_like=螢火蟲,夜景'
                    )
                ]
            ),
            CarouselColumn(
                thumbnail_image_url='https://%s/images/老街.jpg' % server_url,
#                 title='this is menu1',
                text='老街,步道',
                actions=[
                    PostbackAction(
                        label='讚',
#                         text='[::survey_like:]螢火蟲,夜景',
                        data='survey_like=老街,步道'
                    )
                ]
            ),
            CarouselColumn(
                thumbnail_image_url='https://%s/images/推薦.jpg' % server_url,
                text='營地推薦',
                actions=[
                    PostbackAction(
                        label='開始推薦',
#                         text='[::recommend_site:]營地推薦',
                        data='recommend_site'
                    )
                ]
            )
        ]
    )
)

In [ ]:
'''
偏好調查(Carousel Flex)
'''

from linebot.models import(
    FlexSendMessage, CarouselContainer
)

survey_flexCarouselContainerJsonDict ="""
{
    "type": "carousel",
    "contents": [
      {
        "type": "bubble",
        "hero": {
          "type": "image",
          "url": "#img_url_001#",
          "size": "full",
          "aspectMode": "fit"
        },
        "body": {
          "type": "box",
          "layout": "vertical",
          "spacing": "sm",
          "contents": [
            {
              "type": "text",
              "text": "【 填寫問卷】喜歡的就請按，可重複，推薦請按GO",
              "size": "sm",
              "weight": "bold",
              "wrap": true
            }
          ]
        },
        "footer": {
          "type": "box",
          "layout": "vertical",
          "spacing": "sm",
          "contents": [
            {
              "type": "button",
              "action": {
                "type": "postback",
                "label": "沙坑(請按我)",
                "data": "survey_like=沙坑"
              },
              "style": "primary"
            },
            {
              "type": "button",
              "action": {
                "type": "postback",
                "label": "購物(請按我)",
                "data": "survey_like=購物"
              },
              "style": "primary"
            },
            {
              "type": "button",
              "action": {
                "type": "postback",
                "label": "雲海(請按我)",
                "data": "survey_like=雲海"
              },
              "style": "primary"
            },
            {
              "type": "button",
              "action": {
                "type": "postback",
                "label": "螢火蟲(請按我)",
                "data": "survey_like=螢火蟲"
              },
              "style": "primary"
            }
          ]
        }
      },{
        "type": "bubble",
        "hero": {
          "type": "image",
          "url": "#img_url_002#",
          "size": "full",
          "aspectMode": "fit"
        },
        "body": {
          "type": "box",
          "layout": "vertical",
          "spacing": "sm",
          "contents": [
            {
              "type": "text",
              "text": "【 填寫問卷】喜歡的就請按，可重複，推薦請按GO",
              "size": "sm",
              "weight": "bold",
              "wrap": true
            }
          ]
        },
        "footer": {
          "type": "box",
          "layout": "vertical",
          "spacing": "sm",
          "contents": [
            {
              "type": "button",
              "action": {
                "type": "postback",
                "label": "落雨松(請按我)",
                "data": "survey_like=落雨松"
              },
              "style": "primary"
            },
            {
              "type": "button",
              "action": {
                "type": "postback",
                "label": "美食(請按我)",
                "data": "survey_like=美食"
              },
              "style": "primary"
            },
            {
              "type": "button",
              "action": {
                "type": "postback",
                "label": "小溪(請按我)",
                "data": "survey_like=小溪"
              },
              "style": "primary"
            },
            {
              "type": "button",
              "action": {
                "type": "postback",
                "label": "櫻花(請按我)",
                "data": "survey_like=櫻花"
              },
              "style": "primary"
            }
          ]
        }
      },{
        "type": "bubble",
        "hero": {
          "type": "image",
          "url": "#img_url_003#",
          "size": "full",
          "aspectMode": "fit"
        },
        "body": {
          "type": "box",
          "layout": "vertical",
          "spacing": "sm",
          "contents": [
            {
              "type": "text",
              "text": "【 填寫問卷】喜歡的就請按，可重複，推薦請按GO",
              "size": "sm",
              "weight": "bold",
              "wrap": true
            }
          ]
        },
        "footer": {
          "type": "box",
          "layout": "vertical",
          "spacing": "sm",
          "contents": [
            {
              "type": "button",
              "action": {
                "type": "postback",
                "label": "溜滑梯(請按我)",
                "data": "survey_like=溜滑梯"
              },
              "style": "primary"
            },
            {
              "type": "button",
              "action": {
                "type": "postback",
                "label": "山景(請按我)",
                "data": "survey_like=山景"
              },
              "style": "primary"
            },
            {
              "type": "button",
              "action": {
                "type": "postback",
                "label": "海景(請按我)",
                "data": "survey_like=海景"
              },
              "style": "primary"
            },
            {
              "type": "button",
              "action": {
                "type": "postback",
                "label": "油桐花(請按我)",
                "data": "survey_like=油桐花"
              },
              "style": "primary"
            }
          ]
        }
      },{
        "type": "bubble",
        "hero": {
          "type": "image",
          "url": "#img_url_004#",
          "size": "full",
          "aspectMode": "fit"
        },
        "body": {
          "type": "box",
          "layout": "vertical",
          "spacing": "sm",
          "contents": [
            {
              "type": "text",
              "text": "【 填寫問卷】喜歡的就請按，可重複，推薦請按GO",
              "size": "sm",
              "weight": "bold",
              "wrap": true
            }
          ]
        },
        "footer": {
          "type": "box",
          "layout": "vertical",
          "spacing": "sm",
          "contents": [
            {
              "type": "button",
              "action": {
                "type": "postback",
                "label": "湖景(請按我)",
                "data": "survey_like=湖景"
              },
              "style": "primary"
            },
            {
              "type": "button",
              "action": {
                "type": "postback",
                "label": "昆蟲(請按我)",
                "data": "survey_like=昆蟲"
              },
              "style": "primary"
            },
            {
              "type": "button",
              "action": {
                "type": "postback",
                "label": "遮雨棚(請按我)",
                "data": "survey_like=遮雨棚"
              },
              "style": "primary"
            },
            {
              "type": "button",
              "action": {
                "type": "postback",
                "label": "古道(請按我)",
                "data": "survey_like=古道"
              },
              "style": "primary"
            }
          ]
        }
      },{
        "type": "bubble",
        "hero": {
          "type": "image",
          "url": "#img_url_005#",
          "size": "full",
          "aspectMode": "fit"
        },
        "body": {
          "type": "box",
          "layout": "vertical",
          "spacing": "sm",
          "contents": [
            {
              "type": "text",
              "text": "【 填寫問卷】喜歡的就請按，可重複，推薦請按GO",
              "size": "sm",
              "weight": "bold",
              "wrap": true
            }
          ]
        },
        "footer": {
          "type": "box",
          "layout": "vertical",
          "spacing": "sm",
          "contents": [
            {
              "type": "button",
              "action": {
                "type": "postback",
                "label": "高海拔(請按我)",
                "data": "survey_like=高海拔"
              },
              "style": "primary"
            },
            {
              "type": "button",
              "action": {
                "type": "postback",
                "label": "星空(請按我)",
                "data": "survey_like=星空"
              },
              "style": "primary"
            },
            {
              "type": "button",
              "action": {
                "type": "postback",
                "label": "老街(請按我)",
                "data": "survey_like=老街"
              },
              "style": "primary"
            },
            {
              "type": "button",
              "action": {
                "type": "postback",
                "label": "水池(請按我)",
                "data": "survey_like=水池"
              },
              "style": "primary"
            }
          ]
        }
      },
      {
        "type": "bubble",
        "direction": "ltr",
        "hero": {
          "type": "image",
          "url": "#img_url_006#",
          "size": "full",
          "aspectMode": "fit"
        },
        "body": {
          "type": "box",
          "layout": "vertical",
          "contents": [
            {
              "type": "text",
              "text": "營地推薦",
              "align": "center"
            }
          ]
        },
        "footer": {
          "type": "box",
          "layout": "vertical",
           "spacing": "sm",
          "contents": [
            {
              "type": "button",
              "action": {
                "type": "postback",
                "label": "GO",
                "text": "GO",
                "data": "recommend_site"
              },
              "style": "primary"
            },{
              "type": "button",
              "action": {
                "type": "postback",
                "label": "查看點選清單",
                "text": "查看點選清單",
                 "data": "like_site"
              },
              "style": "primary"
            },{
              "type": "button",
              "action": {
                "type": "postback",
                "label": "清空點選清單",
                "text": "清空點選清單",
                "data": "like_site"
              },
              "style": "primary"
            }
          ]
        }
      }
    ]
}
"""


survey_flexCarouselContainerJsonDict = survey_flexCarouselContainerJsonDict.replace("#img_url_001#", ('https://%s/images/0001.jpg' % server_url))
survey_flexCarouselContainerJsonDict = survey_flexCarouselContainerJsonDict.replace("#img_url_002#", ('https://%s/images/0002.jpg' % server_url))
survey_flexCarouselContainerJsonDict = survey_flexCarouselContainerJsonDict.replace("#img_url_003#", ('https://%s/images/0003.jpg' % server_url))
survey_flexCarouselContainerJsonDict = survey_flexCarouselContainerJsonDict.replace("#img_url_004#", ('https://%s/images/0004.jpg' % server_url))
survey_flexCarouselContainerJsonDict = survey_flexCarouselContainerJsonDict.replace("#img_url_005#", ('https://%s/images/0005.jpg' % server_url))
survey_flexCarouselContainerJsonDict = survey_flexCarouselContainerJsonDict.replace("#img_url_006#", ('https://%s/images/0006.jpg' % server_url))
# print(type(survey_flexCarouselContainerJsonDict))
# print(survey_flexCarouselContainerJsonDict)

'''
將carousel類型的json 進行轉換變成 Python可理解之類型物件
將該物件封裝進 Flex Message中
'''
survey_carouselContent = CarouselContainer.new_from_json_dict(json.loads(survey_flexCarouselContainerJsonDict))
survey_flexCarouselSendMeesage = FlexSendMessage(alt_text="偏好調查", contents=survey_carouselContent)


In [ ]:
'''
食譜推薦
'''

from linebot.models import(
    FlexSendMessage, CarouselContainer
)

survey_flexCarouselContainerJsonDict1 ="""
{
    "type": "carousel",
    "contents": [
          {
        "type": "bubble",
        "direction": "ltr",
        "hero": {
          "type": "image",
          "url": "#img_url_007#",
          "size": "full",
          "aspectMode": "fit"
        },
        "body": {
          "type": "box",
          "layout": "vertical",
          "contents": [
            {
              "type": "text",
              "text": "【 填寫問卷】喜歡的就請按，可重複，推薦請按GO",
              "align": "center"
            }
          ]
        },
        "footer": {
          "type": "box",
          "layout": "horizontal",
          "contents": [
            {
              "type": "button",
              "action": {
                "type": "postback",
                "label": "豬肉",
                "text": "豬肉",
                "data": "food_like=豬肉"
              },
              "style": "primary"
            },{
              "type": "button",
              "action": {
                "type": "postback",
                "label": "雞肉",
                "text": "雞肉",
                "data": "food_like=雞肉"
              },
              "style": "primary"
            },{
              "type": "button",
              "action": {
                "type": "postback",
                "label": "牛肉",
                "text": "牛肉",
                "data": "food_like=牛肉"
              },
              "style": "primary"
            }
          ]
        }
      },{
        "type": "bubble",
        "direction": "ltr",
        "hero": {
          "type": "image",
          "url": "#img_url_008#",
          "size": "full",
          "aspectMode": "fit"
        },
        "body": {
          "type": "box",
          "layout": "vertical",
          "contents": [
            {
              "type": "text",
              "text": "喜歡的就請按，可重複",
              "align": "center"
            }
          ]
        },
        "footer": {
          "type": "box",
          "layout": "horizontal",
          "contents": [
            {
              "type": "button",
              "action": {
                "type": "postback",
                "label": "高麗菜",
                "text": "高麗菜",
                "data": "food_like=高麗菜"
              },
              "style": "primary"
            },{
              "type": "button",
              "action": {
                "type": "postback",
                "label": "生菜",
                "text": "生菜",
                "data": "food_like=生菜"
              },
              "style": "primary"
            },{
              "type": "button",
              "action": {
                "type": "postback",
                "label": "空心菜",
                "text": "空心菜",
                "data": "food_like=空心菜"
              },
              "style": "primary"
            }
          ]
        }
      },      {
        "type": "bubble",
        "direction": "ltr",
        "hero": {
          "type": "image",
          "url": "#img_url_009#",
          "size": "full",
          "aspectMode": "fit"
        },
        "body": {
          "type": "box",
          "layout": "vertical",
          "contents": [
            {
              "type": "text",
              "text": "喜歡的就請按，可重複",
              "align": "center"
            }
          ]
        },
        "footer": {
          "type": "box",
          "layout": "horizontal",
          "contents": [
            {
              "type": "button",
              "action": {
                "type": "postback",
                "label": "飯類",
                "text": "飯類",
                "data": "food_like=飯類"
              },
              "style": "primary"
            },{
              "type": "button",
              "action": {
                "type": "postback",
                "label": "粥類",
                "text": "粥類",
                "data": "food_like=粥類"
              },
              "style": "primary"
            },{
              "type": "button",
              "action": {
                "type": "postback",
                "label": "麵類",
                "text": "麵類",
                "data": "food_like=麵類"
              },
              "style": "primary"
            }
          ]
        }
      }, {
        "type": "bubble",
        "direction": "ltr",
        "hero": {
          "type": "image",
          "url": "#img_url_010#",
          "size": "full",
          "aspectMode": "fit"
        },
        "body": {
          "type": "box",
          "layout": "vertical",
          "contents": [
            {
              "type": "text",
              "text": "食譜推薦",
              "align": "center"
            }
          ]
        },
        "footer": {
          "type": "box",
          "layout": "horizontal",
          "contents": [
            {
              "type": "button",
              "action": {
                "type": "postback",
                "label": "GO",
                "text": "GO",
                "data": "recommend_food"
              },
              "style": "primary"
            },{
              "type": "button",
              "action": {
                "type": "postback",
                "label": "查看",
                "text": "查看",
                "data": "like_food"
              },
              "style": "primary"
            },{
              "type": "button",
              "action": {
                "type": "postback",
                "label": "清空",
                "text": "清空",
                "data": "recommend_food"
              },
              "style": "primary"
            }
          ]
        }
      }
    
    ]
}
"""

survey_flexCarouselContainerJsonDict1 = survey_flexCarouselContainerJsonDict1.replace("#img_url_007#", ('https://%s/images/0007.jpg' % server_url))
survey_flexCarouselContainerJsonDict1 = survey_flexCarouselContainerJsonDict1.replace("#img_url_008#", ('https://%s/images/0008.jpg' % server_url))
survey_flexCarouselContainerJsonDict1 = survey_flexCarouselContainerJsonDict1.replace("#img_url_009#", ('https://%s/images/0009.jpg' % server_url))
survey_flexCarouselContainerJsonDict1 = survey_flexCarouselContainerJsonDict1.replace("#img_url_010#", ('https://%s/images/0006.jpg' % server_url))
# print(type(survey_flexCarouselContainerJsonDict))
# print(survey_flexCarouselContainerJsonDict)

'''
將carousel類型的json 進行轉換變成 Python可理解之類型物件
將該物件封裝進 Flex Message中
'''
survey_carouselContent1 = CarouselContainer.new_from_json_dict(json.loads(survey_flexCarouselContainerJsonDict1))
survey_flexCarouselSendMeesage1 = FlexSendMessage(alt_text="偏好調查", contents=survey_carouselContent1)

In [ ]:
#botton3搜尋-選擇地區
#用圖片(要是網址)使用動作，回傳文字、回傳網站

from linebot.models import ImagemapSendMessage

from linebot.models import (
    ImagemapArea, BaseSize, URIImagemapAction, MessageImagemapAction
)

imagemap_message = ImagemapSendMessage(
    base_url='https://is.gd/6CEqzq',
    alt_text='選擇區域',
    base_size=BaseSize(height=1364,width=1040)
     ,actions=[
        MessageImagemapAction(
            text="[::text:]北部地區",
            area=ImagemapArea(
                x=271, y=152, width=653,height=262
            )
        ),
         MessageImagemapAction(
            text="[::text:]中部地區",
            area=ImagemapArea(
                x=45, y=426,width=598,height=366
            )
        ),
         MessageImagemapAction(
            text="[::text:]南部地區",
            area=ImagemapArea(
                x=16, y=812,width=504,height=511
            )
        ),
         MessageImagemapAction(
            text="[::text:]東部地區",
            area=ImagemapArea(
                x=648, y=562,width=390,height=733
            )
        )
        
    ]
) 


In [ ]:


from linebot.models.template import(
    ButtonsTemplate
)

itemidentify_buttons_template_message = TemplateSendMessage(
    alt_text = '器材辨識',
    template = ButtonsTemplate(
        title = '器材辨識',
        text = '選擇手機照片或拍照後再上傳',
        actions = [
          {
            "type": "uri",
            "label": "拍張照片",
            "uri": "line://nv/camera/"
          }
        ]
    )
)

In [ ]:
'''
將素材消息做成一本字典，當用戶發出相應消息時，可從此進行查找動作。
'''

template_message_dict = {
#     "[::text:]營地推薦": survey_carousel_template_message
     "[::text:]營地推薦":[survey_flexCarouselSendMeesage]
    ,"[::text:]營地搜尋":imagemap_message
    ,"[::text:]器材辨識":itemidentify_buttons_template_message
    ,"[::text:]食譜推薦":survey_flexCarouselSendMeesage1
#    ,"[::text:]查看清單":
#     , "[::text:]清空清單":image_message

}

#     , "[::text:]營地搜尋":image_message
#     , "[::text:]器材辨識":image_message2
#     , "[::text:]關於我們":image_message3

In [ ]:
'''

當用戶發出文字消息時，判斷文字內容是否包含[::text:]，
    若有，則從template_message_dict 內找出相關訊息
    若無，則回傳預設訊息。

'''

from linebot.models import (
    MessageEvent, TextMessage, TextSendMessage,
)

# 用戶發出文字消息時， 按條件內容, 回傳文字消息
@handler.add(MessageEvent, message=TextMessage)
def handle_message(event):
    print("event.message.text:", event.message.text)
    user_profile = line_bot_api.get_profile(event.source.user_id)
    user_profile_food = line_bot_api.get_profile(event.source.user_id)
    if (event.message.text.find('::text:') != -1):
        if "[::text:]營地推薦" == event.message.text:
            del_user_prop(user_profile, "survey_like")
        elif "[::text:]食譜推薦" == event.message.text:
            del_user_prop_food(user_profile_food, "food_like")
        elif "[::text:]器材辨識" == event.message.text:
            put_user_prop(user_profile, "do_itemidentify", True)
        line_bot_api.reply_message(
            event.reply_token,
            template_message_dict.get(event.message.text))
    #?????????????
    elif (event.message.text.find('::food_like:') != -1):
        pass   
    elif (event.message.text.find('::survey_like:') != -1):
        pass
#         print("偏好調查(by TextMessage):", event.message.text.replace("[::survey_like:]", "").replace(",", " "))
    else:
        pass
#         line_bot_api.reply_message(
#         event.reply_token,
#         TextSendMessage(text="請點擊菜單上圖面，或輸入[::text:]more，取得更多幫助"))

In [ ]:
#用戶點擊button後，觸發postback event，對其回傳做相對應處理

from linebot.models import (
    PostbackEvent
)

'''
user_profile: {"displayName": "Nelson Chou"
, "pictureUrl": "https://profile.line-scdn.net/0hu_yDKA7dKnBEPQC6sJZVJ3h4JB0zEyw4PA5tHjZpIRBsBD0gLA9sEGA1cUVoWTh0fwg1RWBqIUdr"
, "userId": "U8633692756652c60054e28e65760bf8b"}
'''

@handler.add(PostbackEvent)
def handle_post_message(event):
    user_profile = line_bot_api.get_profile(event.source.user_id)
    #點選人的ID
    print("userId:", user_profile.user_id)
    #顯示回傳值
    print("event.postback.data:", event.postback.data)
    #如果回傳值是沒有按過的
    if (event.postback.data.find('survey_like') == 0):
        #第一次將回傳值帶入survey_like轉成list、並加上逗號
        survey_like = event.postback.data.replace("survey_like=", "").split(",")
        #顯示list
        print("營地偏好調查(by PostbackEvent):", survey_like)
        # get_user_prop上面有方法解釋
        #把舊的資訊拿出來
        ori_survey_like = get_user_prop(user_profile, "survey_like")
        #如果舊的有資訊
        if ori_survey_like:
            #回傳值變成一個list，去除重複值   
            #並增家後面的回傳值
            survey_like = list(set(survey_like).union(ori_survey_like))
        #創進依一個新的
        put_user_prop(user_profile, "survey_like", survey_like)
        #顯示目前字典裡的內容
        print("survey_like:", get_user_prop(user_profile, "survey_like"))
    
    #如果還沒有按過營地推薦，執行以下內容
    if (event.postback.data.find('survey_site') == 0):
        #顯示目前字典裡的內容
        survey_like = get_user_prop(user_profile, "survey_like")
        print("survey_like:", survey_like)
        #如果還沒有數入回傳值，顯示回復訊息:請先讚過示意圖片!!!
        if survey_like == None:
            line_bot_api.reply_message(
                event.reply_token,
                TextSendMessage("請先讚過示意圖片!!!"))
        #如果有回傳值，先抓到
        else:
            survey_like = None
    
    if (event.postback.data.find('like_site') == 0):
        #顯示目前字典裡的內容
        survey_like = get_user_prop(user_profile, "survey_like")
        print("survey_like:", survey_like)
        #如果還沒有數入回傳值，顯示回復訊息:請先讚過示意圖片!!!
        if survey_like == None:
            line_bot_api.reply_message(
                event.reply_token,
                TextSendMessage("請先讚過示意圖片!!!"))
        #如果有回傳值，顯示回復訊息:以輸入過回傳值
        else:
            print("使用者選擇查看清單...")
            survey_like = get_user_prop(user_profile, "survey_like")
            print("result:", survey_like)
            line_bot_api.reply_message(
                event.reply_token,
                TextSendMessage(",".join(survey_like)))
            
    #如果還沒有按過營地推薦，執行以下內容
    if (event.postback.data.find('recommend_site') == 0):
        #顯示目前字典裡的內容
        survey_like = get_user_prop(user_profile, "survey_like")
        print("survey_like:", survey_like)
        #如果還沒有數入回傳值，顯示回復訊息:請先讚過示意圖片!!!
        if survey_like == None:
            line_bot_api.reply_message(
                event.reply_token,
                TextSendMessage("請先讚過示意圖片!!!"))
        #如果有回傳值，先抓到
        else:          
            #title,addr,url連在一起(測試)\n",
            recommend_camp_sites_info= recommend_camp_sites(" ".join(survey_like), 3)
            result_camp_title = recommend_camp_sites_info['camp_title']
            result_addr = recommend_camp_sites_info['addr']
            result_url = recommend_camp_sites_info['url']
            result__score= recommend_camp_sites_info['score']
            print("result:",  result_camp_title)
            print("result:", result_addr)
            print("result:", result_url)
            print("result:", result__score)

            
#             result = result.append(result1).append(result2)
#             result_final=TextSendMessage("\n".join(result))
#             print("\n".join(result))
#             print(type("\n".join(result)))
#             line_bot_api.reply_message(event.reply_token,result_final)
           
            result_final=[]
            for n in range(0,3):
                result_list=[]
                result_list.append('【名稱】'+list(result_camp_title.items())[n][1])
                #result_list.append(list(result_camp_title.items())[n][1])
                result_list.append('【地址】'+list(result_addr.items())[n][1])
                #result_list.append(list(result_addr.items())[n][1])
                result_list.append('【網址】'+list(result_url.items())[n][1])
                #result_list.append(list(result_url.items())[n][1])
                result_list=TextSendMessage("\n".join(result_list))
                result_final.append(result_list)
            line_bot_api.reply_message(event.reply_token,result_final)
            
            
#食鋪推薦            


            
    user_profile_food = line_bot_api.get_profile(event.source.user_id)
    #點選人的ID
    print("userId:", user_profile_food.user_id)
    #顯示回傳值
    print("event.postback.data:", event.postback.data)
    #如果回傳值是沒有按過的
    if (event.postback.data.find('food_like') == 0):
        #第一次將回傳值帶入food_like轉成list、並加上逗號
        food_like = event.postback.data.replace("food_like=", "").split(",")
        #顯示list
        print("食譜偏好調查(by PostbackEvent):", food_like)
        # get_user_prop上面有方法解釋
        #把舊的資訊拿出來
        ori_food_like = get_user_prop_food(user_profile_food, "food_like")
        #如果舊的有資訊
        if ori_food_like:
            #回傳值變成一個list，去除重複值   
            #並增家後面的回傳值
            food_like = list(set(food_like).union(ori_food_like))
        #創進依一個新的
        put_user_prop_food(user_profile_food, "food_like", food_like)
        #顯示目前字典裡的內容
        print("food_like:", get_user_prop_food(user_profile_food, "food_like"))
    
    #如果還沒有按過營地推薦，執行以下內容
    if (event.postback.data.find('clean_food') == 0):
        #顯示目前字典裡的內容
        food_like = get_user_prop_food(user_profile_food, "food_like")
        print("food_like:", food_like)
        #如果還沒有數入回傳值，顯示回復訊息:請先讚過示意圖片!!!
        if food_like == None:
            line_bot_api.reply_message(
                event.reply_token,
                TextSendMessage("請先讚過示意圖片!!!"))
        #如果有回傳值，先抓到
        else:
            food_like = None
    
    if (event.postback.data.find('like_food') == 0):
        #顯示目前字典裡的內容
        food_like = get_user_prop_food(user_profile_food, "food_like")
        print("food_like:", food_like)
        #如果還沒有數入回傳值，顯示回復訊息:請先讚過示意圖片!!!
        if food_like == None:
            line_bot_api.reply_message(
                event.reply_token,
                TextSendMessage("請先讚過示意圖片!!!"))
        #如果有回傳值，顯示回復訊息:以輸入過回傳值
        else:
            print("使用者選擇查看清單...")
            food_like = get_user_prop_food(user_profile_food, "food_like")
            print("result:", food_like)
            line_bot_api.reply_message(
                event.reply_token,
                TextSendMessage(",".join(food_like)))
            
    #如果還沒有按過營地推薦，執行以下內容
    if (event.postback.data.find('recommend_food') == 0):
        #顯示目前字典裡的內容
        food_like = get_user_prop_food(user_profile_food, "food_like")
        print("food_like:", food_like)
        #如果還沒有數入回傳值，顯示回復訊息:請先讚過示意圖片!!!
        if food_like == None:
            line_bot_api.reply_message(
                event.reply_token,
                TextSendMessage("請先讚過示意圖片!!!"))
        #如果有回傳值，先抓到
        else:
            pass
            
            ##title,addr,url連在一起(測試)
            #result = recommend_camp_sites(" ".join(food_like), 1)['camp_title']
            ##a=[TextSendMessage("\n".join(result)),TextSendMessage("\n".join(result1))]
            #line_bot_api.reply_message(
                #event.reply_token,
                #TextSendMessage("\n".join(result)))
            

          

In [ ]:
'''

若收到圖片消息時，

先回覆用戶文字消息，並從Line上將照片拿回。

'''
from linebot.models import (
    ImageMessage
)

@handler.add(MessageEvent, message=ImageMessage)
def handle_message(event):
    user_profile = line_bot_api.get_profile(event.source.user_id)
    if get_user_prop(user_profile, "do_itemidentify"):
        message_content = line_bot_api.get_message_content(event.message.id)
        img_path = './upload_images/'+event.message.id+'.jpg'
        with open(img_path, 'wb') as fd:
            for chunk in message_content.iter_content():
                fd.write(chunk)
        item_name = cnn_itemindntify(img_path)
        print("item_name =", item_name, ", type =", type(item_name))
        put_user_prop(user_profile, "do_itemidentify", False)
        line_bot_api.reply_message(
                event.reply_token,
                TextSendMessage("這是~{}".format(item_name)))


In [ ]:
'''

執行此句，啟動Server，觀察後，按左上方塊，停用Server

'''

if __name__ == "__main__":
    app.run(host='0.0.0.0')

 * Running on http://0.0.0.0:5000/ (Press CTRL+C to quit)


event.message.text: [::text:]營地推薦


172.17.0.1 - - [24/May/2019 12:25:38] "POST / HTTP/1.1" 200 -


userId: U873be787f1bc8a4322527ab2310bc0ab
event.postback.data: survey_like=雲海
營地偏好調查(by PostbackEvent): ['雲海']
survey_like: ['雲海']


172.17.0.1 - - [24/May/2019 12:25:41] "POST / HTTP/1.1" 200 -


...

userId: U873be787f1bc8a4322527ab2310bc0ab
event.postback.data: survey_like=螢火蟲
營地偏好調查(by PostbackEvent): ['螢火蟲']
survey_like: ['螢火蟲']


172.17.0.1 - - [24/May/2019 12:27:03] "POST / HTTP/1.1" 200 -


userId: U873be787f1bc8a4322527ab2310bc0ab
event.postback.data: survey_like=螢火蟲
userId: U873be787f1bc8a4322527ab2310bc0ab
event.postback.data: survey_like=雲海
營地偏好調查(by PostbackEvent): ['雲海']
survey_like: ['螢火蟲', '雲海']


172.17.0.1 - - [24/May/2019 12:27:07] "POST / HTTP/1.1" 200 -


userId: U873be787f1bc8a4322527ab2310bc0ab
event.postback.data: survey_like=雲海
